In [ ]:
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function


import nltk, re, pprint
from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.corpus import wordnet as wn
import io

import urllib
from urllib import urlopen
from bs4 import BeautifulSoup
from textwrap import fill
import numpy as np#for math processing
import matplotlib #for graphs
from numpy import arange #to handle ranges
from matplotlib import pyplot #plotting 
import networkx as nx # brings up node configs

import unicodedata
import random
from collections import defaultdict
import csv
import pandas
import xlrd
def __unicode__(self):

    return  u''

In [ ]:
df = pandas.read_excel('filename')
df

In [ ]:
df.columns = ["PAT_DEID",
"NOTE_DEID",
"NOTE",
"NOTE_CODE",
u'NLPGeneral', u'NLPRegional', u'NLPLocal', u'ExpertGeneral', u'ExpertRegional',
       u'ExpertLocal']



In [ ]:
df = df.drop(df.index[0])
df.reset_index()

In [ ]:
print(df.columns)
for column in df.columns[4:10]:
    df[column] = df[column].astype(int) 
df["NLPGeneral"][96]

In [ ]:
import random
r = random.sample(range(600, 800), 100)
p = random.sample(range(0,1000), 100)
len(df)
df["NOTE_DEID"] = r
df["PAT_DEID"] = p

In [ ]:
#for hierarchical indexing:
df2 = df[[
    u'NLPGeneral',
    u'NLPRegional', u'NLPLocal', u'ExpertGeneral', u'ExpertRegional',
       u'ExpertLocal']]
tup = [("NLP","General"), ("NLP","Regional"), ("NLP","Local"), ("Expert","General"), ("Expert","Regional"), ("Expert","Local")]
columns = pandas.MultiIndex.from_tuples(tup)
df2.columns = columns

In [ ]:
# This creates a third dataframe where the overall values are stored -- is needed to keep track of which rows pertain to which surgery/note
comparison1 = pandas.DataFrame()
comparison1["Index"] = df.index
comparison1["NOTE_CODE"] = df["NOTE_CODE"]
comparison1["General"] = np.nan
comparison1["Regional"] = np.nan
comparison1["Local"] = np.nan
comparison1["Overall"] = np.nan

In [ ]:
#combined.reset_index()
#combined.index = range(88)
combined= df

# stores all the accuracy values in a dataframe that has been sorted by type of operation
for i in range(1, len(combined)):
    if combined.loc[i, "NLPGeneral"] == 1 & combined.loc[i, "ExpertGeneral"] == 1:
        comparison1.loc[i,"General"] = 1
    if combined.loc[i, "NLPRegional"] == 1 & combined.loc[i, "ExpertRegional"] == 1:
        comparison1.loc[i,"Regional"] = 1
    if combined.loc[i, "NLPLocal"] == 1 & combined.loc[i, "ExpertLocal"] == 1:
        comparison1.loc[i,"Local"] = 1
    comparison1.loc[i, "Overall"] = comparison1.loc[i,"General"] + comparison1.loc[i,"Regional"] + comparison1.loc[i,"Local"]
comparison1 = comparison1.fillna(0)


combined["NLPOVERALL"] = np.nan
combined["ExpertOverall"] = np.nan
for i in range(1, len(combined)+1):
    
    combined.loc[i, "NLPOVERALL"] = (combined.loc[i, "NLPGeneral"]) + (combined.loc[i, "NLPRegional"]) + combined.loc[i, "NLPLocal"]
    combined.loc[i, "ExpertOverall"] = combined.loc[i, "ExpertGeneral"] + combined.loc[i, "ExpertRegional"] + combined.loc[i, "ExpertLocal"]

In [ ]:
G_auto = pandas.Series(list(combined["NLPGeneral"].values))
G_clini = pandas.Series(list(combined["ExpertGeneral"].values))
General_CF = pandas.crosstab(G_clini, G_auto, rownames=['True'], colnames=['Predicted'], margins=True)

###
R_auto = pandas.Series(list(combined["NLPRegional"].values))
R_clini = pandas.Series(list(combined["ExpertRegional"].values))
Regional_CF = pandas.crosstab(R_clini, R_auto, rownames=['True'], colnames=['Predicted'], margins=True)

###
L_auto = pandas.Series(list(combined["NLPLocal"].values))
L_clini = pandas.Series(list(combined["ExpertLocal"].values))
Local_CF = pandas.crosstab(L_clini, L_auto, rownames=['True'], colnames=['Predicted'], margins=True)

###

O_auto = pandas.Series(list(combined["NLPOVERALL"].values))
O_clini = pandas.Series(list(combined["ExpertOverall"].values))
Overall_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
General_PPV = General_CF.loc[1,1]
General_False_Pos = General_CF.loc[0,1]
General_NPV = General_CF.loc[0,0]
General_False_Neg = General_CF.loc[1,0]

Regional_PPV = Regional_CF.loc[1,1]
Regional_False_Pos = Regional_CF.loc[0,1]
Regional_NPV  = Regional_CF.loc[0,0]
Regional_False_Neg = Regional_CF.loc[1,0]

Local_PPV = Local_CF.loc[1,1]
Local_False_Pos = Local_CF.loc[0,1]
Local_NPV =  Local_CF.loc[0,0]
Local_False_Neg = Local_CF.loc[1,0]

Overall_PPV = Overall_CF.loc[1,1] + Overall_CF.loc[2,2]
Overall_False_Pos = Overall_CF.loc[0,1] + Overall_CF.loc[0,2] + Overall_CF.loc[1,2]
Overall_NPV = Overall_CF.loc[0,0] #fine
Overall_False_Neg = Overall_CF.loc[1,0] + Overall_CF.loc[2,0] + Overall_CF.loc[2,1]


In [ ]:
results = pandas.DataFrame({'class': ['General',"Regional","Local", "Overall"],
                        'tp': [General_PPV, Regional_PPV, Local_PPV, Overall_PPV],
                        'fp': [General_False_Pos, Regional_False_Pos, Local_False_Pos, Overall_False_Pos],
                        'tn': [General_NPV, Regional_NPV, Local_NPV, Overall_NPV],
                        'fn': [General_False_Neg, Regional_False_Neg, Local_False_Neg, Overall_False_Neg]
                       })

In [ ]:
def calculate_metrics(results):
    roc = pandas.DataFrame(index = results['class'], columns=['recall', 'precision', 'f1', 'tpr', 'fpr'])
    
    for i in results.iterrows():
        t, fn, fp, tn, tp = i[1]
        

        print(int(tp) + int(fp) + int(tn) + int(fn))
        assert int(tp) + int(fp) + int(tn) + int(fn) == 25 or int(tp) + int(fp) + int(tn) + int(fn) == 100 , 'must equal length of dataset or else'
        
        recall = tp / (tp + fn)
        
        if tp == fp == 0:
            precision = 0
            true_positive_rate = 0
        
        else:
            precision = tp / (tp + fp)
            true_positive_rate = tp / (tp + fn)
            
        if precision == recall == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        
        if fp + tn == 0:
            
            false_positive_rate =0
        else:
            false_positive_rate = fp / (fp + tn)
        
        roc.loc[t, 'recall'] = recall
        roc.loc[t, 'precision'] = precision
        roc.loc[t, 'f1'] = f1
        roc.loc[t, 'tpr'] = true_positive_rate
        roc.loc[t, 'fpr'] = false_positive_rate
    
    return roc

In [ ]:
roc = calculate_metrics(results)
roc.reset_index()

In [ ]:
stats2 = pandas.DataFrame()
stats1 = combined.groupby('NOTE_CODE').count()
stats2["General"] = np.nan
stats2["Regional"] = np.nan
stats2["Local"] = np.nan
stats2["Count"] = [value for value in list(stats1["NOTE_DEID"].values)]
stats2["Percent"] = np.nan

In [ ]:
notemismatch = random.sample(range(600,650), 50)
print(notemismatch)
(df.loc[df['NOTE_DEID'].isin(notemismatch)])

In [ ]:
stats2.index = [index for index in stats1.index]

In [ ]:
for row in stats2.index:
    df1 = comparison1.loc[comparison1['NOTE_CODE'] == row]
    #stats2.at(i, "General") = 0
    #print(str("General ") + str(sum(df1["General"].values))) #df1 takes its values from comparison1, so its values are already finalized/have been compared
    stats2.loc[row, 'General'] = sum(df1["General"].values)
    
    stats2.loc[row, 'Regional'] = sum(df1["Regional"].values)
    
    stats2.loc[row, 'Local'] = sum(df1["Local"].values)
    
    stats2.loc[row,"Percent"] = max(sum(df1["General"].values), sum(df1["Regional"].values), sum(df1["Local"].values))/stats2.loc[row, 'Count']


In [ ]:
import matplotlib

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:


df_O = pandas.DataFrame(Overall_CF/len(comparison1), range(3),
                  range(3))

sn.set(font_scale=1.4)#for label size
sn.heatmap(df_O, annot=True,annot_kws={"size": 16}, cmap="Blues")# font size


In [ ]:


df_G = pandas.DataFrame(General_CF, range(2),
                  range(2))


sn.set(font_scale=1.4)#for label size
sn.heatmap(df_G, annot=True,annot_kws={"size": 16}, cmap="Blues")# font size


In [ ]:


df_R = pandas.DataFrame(Regional_CF, range(2),
                  range(2))


sn.set(font_scale=1.4)#for label size
sn.heatmap(df_R, annot=True,annot_kws={"size": 16}, cmap="Greens")# font size

In [ ]:

df_L = pandas.DataFrame(Local_CF, range(2),
                  range(2))


sn.set(font_scale=1.4)#for label size
sn.heatmap(df_L, annot=True,annot_kws={"size": 16}, cmap="Purples", fmt='g')# font size

In [ ]:
#wht levels of pain are there?
# maybe tally up overlaps (like single out ones with multiple anesthesia types)
# Could potentially count patients with mutliple operations, but there runs the risk of getting patients with 
# consider doing both 30days - baseline and 30 days only, see which is more telling?
#run through and append a dataframe based on note ID
# general distributions


In [ ]:
stats2["Multiple Types Count"] = np.nan


In [ ]:
counts = []
for row in stats2.index:
    count =  0
    df1 = combined.loc[combined['NOTE_CODE'] == row]
    df1.reset_index()
    for i in range(df1.index[0], len(df1)+df1.index[0]):
        if df1.loc[i, "Overall"] > 1:
            #stats2.loc[row, "Multiple Types Count"] = stats2.loc[row, "Multiple Types Count"] +1
            count = count +1
    counts.append(count)
        

In [ ]:
stats2["Multiple Types Count"] = [count for count in counts]

In [ ]:
surgerytypes= combined.groupby('NOTE_CODE').count()

In [ ]:

df_O = pandas.DataFrame(Overall_CF/len(comparison1), range(3),
                  range(3))

sn.set(font_scale=1.4)#for label size
sn.heatmap(df_O, annot=True,annot_kws={"size": 16}, cmap="Blues")# font size


In [ ]:

stats2
stats2["type"] = [type for type in stats2.index]
stats2["General%"] = np.nan
stats2["Regional%"] = np.nan
stats2["Local%"] = np.nan
for index, row in stats2.iterrows():
    stats2.loc[index, "General%"] = stats2.loc[index, "General"]/stats2.loc[index, "Count"] * 100
    stats2.loc[index, "Regional%"] = stats2.loc[index, "Regional"]/stats2.loc[index, "Count"] *100 
    stats2.loc[index, "Local%"] = stats2.loc[index, "Local"]/stats2.loc[index, "Count"] * 100
    
stats2["Multiple Types Count"] = np.nan
counts = []
for row in stats2.index:
    count =  0
    df1 = combined.loc[combined['NOTE_CODE'] == row]
    df1.reset_index()
    for i in range(df1.index[0], len(df1)+df1.index[0]):
        if df1.loc[i, "Overall"] > 1:
            #stats2.loc[row, "Multiple Types Count"] = stats2.loc[row, "Multiple Types Count"] +1
            count = count +1
    counts.append(count)
stats2["Multiple Types Count"] = [count for count in counts]

#sn.barplot(stats2.index,  stats2.General stats2.Regional , data=stats2, palette="Blues_d")

In [ ]:
data1 = stats2[["General%","Regional%","Local%","type"]]
data1.columns = ["General", "Regional","Local","Note Code"]
data1m = data1.melt('Note Code', var_name='Anesthesia Type', value_name='Percent')
data2 = stats2[["General","Regional","Local","type"]]

data1m = data1m.sort_values(by=['Note Code'])


In [ ]:
import seaborn as sns

cmap = sns.color_palette("GnBu_r")#sns.cubehelix_palette()   
sns.set_style('ticks')
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(15, 5)
counts = sns.barplot(x='Note Code', y='Percent', hue='Anesthesia Type', data=data1m, palette=cmap)
new_title = 'Anesthesia Type Frequency vs. Type of Note'
counts.set_title(new_title)

    
plt.show()
# replace labels

In [ ]:
#     
df_O = pandas.DataFrame(Overall_CF, range(1,3),
                  range(1,3))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_O, annot=True,annot_kws={"size": 16}, cmap="Blues", fmt='g')# font size

In [ ]:
roc2 = roc.melt('type1', var_name='metric', value_name='value')

In [ ]:
roc["type1"] = [value for value in roc.index]

In [ ]:
roc

In [ ]:
df1 = combined.loc[combined['NOTE_CODE'] == "T.10003"]
df1.reset_index()
O_auto = pandas.Series(list(df1["Overall"].values))
O_clini = pandas.Series(list(df1["overall"].values))
Note1_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)  

df1 = combined.loc[combined['NOTE_CODE'] == "T.10004"]
df1.reset_index()
O_auto = pandas.Series(list(df1["Overall"].values))
O_clini = pandas.Series(list(df1["overall"].values))
Note2_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)  

df1 = combined.loc[combined['NOTE_CODE'] == "I.28"]
df1.reset_index()
O_auto = pandas.Series(list(df1["Overall"].values))
O_clini = pandas.Series(list(df1["overall"].values))
Note3_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)  

#for additional types of notes. 
df1 = combined.loc[combined['NOTE_CODE'] == ""]
df1.reset_index()
O_auto = pandas.Series(list(df1["Overall"].values)) # the machine
O_clini = pandas.Series(list(df1["overall"].values)) # clinician 
Note4_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)  

# val = [PPV, False Pos, NPV, False Neg], update coordinates
Note1val = [Note1_CF.loc[2,2]+ Note1_CF.loc[1,1], Note1_CF.loc[1,2], 0 , Note1_CF.loc[2,1]]
Note2val = [Note2_CF.loc[2,2]+Note2_CF.loc[1,1], Note2_CF.loc[1,2], 0, Note2_CF.loc[2,1]]
Note3val = [Note3_CF.loc[2,2]+Note3_CF.loc[1,1],  Note3_CF.loc[1,2], 0, Note3_CF.loc[2,1]]
Note4val = [Note4_CF.loc[1,1], Note4_CF.loc[0,1], Note4_CF.loc[0,0],Note4_CF.loc[1,0]]


O_auto = pandas.Series(list(combined["Overall"].values))
O_clini = pandas.Series(list(combined["overall"].values))
Overall_CF = pandas.crosstab(O_clini, O_auto, rownames=['True'], colnames=['Predicted'], margins=True)


Overallval = [Overall_CF.loc[1,1] + Overall_CF.loc[2,2], Overall_CF.loc[0,1] + Overall_CF.loc[0,2] + Overall_CF.loc[1,2],
              Overall_CF.loc[0,0], Overall_CF.loc[1,0] + Overall_CF.loc[2,0] + Overall_CF.loc[2,1]]


#change these to note codes. 

resultsS = pandas.DataFrame({'metric': ['tp','fp','tn','fn'],
                        'DISTAL RADIAL FRACTURE': [value for value in Note1val],
                        'KNEE REPLACEMENT': [value for value in Note2val],
                        'MASTECTOMY': [value for value in Note3val],
                        'THORACOTOMY': [value for value in Note4val],
                        'Overall': [value for value in Overallval]
                       })


In [ ]:
resultsS = resultsS.transpose()
resultsS

In [ ]:
resultsS.columns = ['tp','fp','tn','fn']

In [ ]:
resultsS = resultsS.drop('metric')

In [ ]:
resultsS["class"] = [val for val in resultsS.index]

In [ ]:

roc = calculate_metrics(resultsS)
roc.reset_index()

In [ ]:
resultsS = resultsS[['class', 'tp',
'fp',
'tn',
'fn',
]]

In [ ]:
results

In [ ]:

roc = calculate_metrics(resultsS)
roc.reset_index()